# Creating a Neural Network to Predict Arrests

### By: ZXS107020

In this exercise, we will create an artificial neural network to predict arrests based on crime descriptions provided by the Chicago Police Department.

The data can be downloaded from [here](https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-present/ijzp-q8t2). 

The dataset is quite large(over 1.5 Gb), making it a rich target for machine learning.  

In order to construct our neural network, we will rely on the `keras` package in python. For a more in-depth article on the theory behind the network- [here](https://www.springboard.com/blog/beginners-guide-neural-network-in-python-scikit-learn-0-18/) is a great article.  

In any case, lets get started.

---

## Pre-processing:  

In order to be able to work with this file. We will first have to do quite a bit of pre-processing in order to prepare the material for ingestion. 

1) Convert the data- The file contains a mixture of numeric and categorical variables. We will have to convert these to formats that be understood by our machine learning package. For this process, we will use the `zxs` package that I have created. 

More information on the `zxs` package can be found through this [link](www.github.com/zxs107020/zxs). 

If you are on a Mac, the process is quite simple. Simply open up a terminal window and type:
- `pip install zxs` (obviously, you will first have to install pip as well)  

I have already prepared the numeric file. For the sake of brevity, I will skip the explanation of how this was done.  

2) Prepare the environment for `keras`- this package uses a "backend" that can be one of many other machine learning packages in Python, such as : scikit-learn, theano, or tensorflow. For our purposes, we will use tensorflow as the backend.  

3) Once the environment has been prepared, you will be ready to install `keras`. If you are on a mac, simply open up a terminal window and type:
* `pip install keras`

---

## Machine Learning:

Now that we have set-up our environment and installed our dependencies, we are ready to dive into the machine learning.

In [1]:
# Import the required libraries
import zxs
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Set seed
np.random.seed(100)

In [3]:
# Define workspace
wd = '/path/to/files'
fn = 'numeric_file.csv'

In [6]:
# Import data
data = zxs.local_import(wd, fn)

In [7]:
# Drop index and date
drop_cols = [0,1]
data.drop(data.columns[drop_cols], axis = 1, inplace = True)

In [8]:
# Convert to numeric
data.apply(pd.to_numeric)

,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year
1,36292,30,2,40,155,0,0,131,11,21,19,5,1144606.0,1903566.0,14
2,46997,359,25,257,155,1,0,82,7,7,65,25,1166468.0,1860715.0,14
3,34340,52,2,137,18,0,1,18,2,34,34,10,1185075.0,1875622.0,14
4,38473,40,2,315,18,0,0,21,2,23,36,10,1178033.0,1870804.0,14
5,36206,18,30,53,151,0,0,133,11,21,19,3,1144920.0,1898709.0,14
6,37187,40,2,315,18,0,0,19,2,34,34,10,1183018.0,1872537.0,14
7,48518,52,2,137,18,0,1,86,7,9,66,10,1170859.0,1858210.0,14
8,33202,52,2,137,18,0,1,14,2,23,33,10,1178746.0,1876914.0,14
9,29476,257,18,273,155,1,0,105,9,3,56,20,1164279.0,1880656.0,14
10,55338,40,2,315,136,0,0,53,5,47,45,10,1179637.0,1840444.0,14


In [9]:
# Check for missing values
data.isnull().sum()

0
Block                   0
IUCR                    0
Primary Type            0
Description             0
Location Description    0
Arrest                  0
Domestic                0
Beat                    0
District                0
Ward                    0
Community Area          0
FBI Code                0
X Coordinate            0
Y Coordinate            0
Year                    0
dtype: int64

In [10]:
# Fill missing Values
data.fillna({'X Coordinate': 0, 'Y Coordinate': 0}, inplace = True)

In [11]:
# Split the data for training/validation/testing
x_train, x_val, x_test, y_train, y_val, y_test = zxs.separate(data, 'Arrest', .3)

/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [12]:
# Check the data types
data.dtypes

0
Block                   object
IUCR                    object
Primary Type            object
Description             object
Location Description    object
Arrest                  object
Domestic                object
Beat                    object
District                object
Ward                    object
Community Area          object
FBI Code                object
X Coordinate            object
Y Coordinate            object
Year                    object
dtype: object

In [13]:
# Convert to numeric format
data = data.apply(pd.to_numeric)

In [14]:
# Check for missing values
data.isnull().sum()

0
Block                       0
IUCR                        0
Primary Type                0
Description                 0
Location Description        0
Arrest                      0
Domestic                    0
Beat                        0
District                    0
Ward                        0
Community Area              0
FBI Code                    0
X Coordinate            56828
Y Coordinate            56828
Year                        0
dtype: int64

In [15]:
# Fill missing Values
data.fillna({'X Coordinate': 0, 'Y Coordinate': 0}, inplace = True)

In [16]:
# Split the data for training/validation/testing
x_train, x_val, x_test, y_train, y_val, y_test = zxs.separate(data, 'Arrest', .3)

Now, its time for the heavy lifting. We are going to create a function to build our neural network. For more information on the details that will go into building this model, please see the previously referenced article on ANN's. 

In [17]:
# Create a model
def model():
    model = Sequential()
    model.add(Dense(14, input_dim = 14, kernel_initializer = 'normal', activation = 'relu'))
    model.add(Dense(1, kernel_initializer = 'normal', activation = 'sigmoid'))
    model.add(Dropout(.25))	
    
    # Compile model
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    
    return model

In [18]:
# Create a pipeline
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn = model, epochs = 1, batch_size = 5, verbose = 1)))
pipeline = Pipeline(estimators)

In [19]:
# Run the model
pipeline.fit(x_train, y_train)

Epoch 1/1
4582829/4582829 [==============================] - 1626s 355us/step - loss: 1.4068 - acc: 0.8276


Pipeline(memory=None,
     steps=[('standardize', StandardScaler(copy=True, with_mean=True, with_std=True)), ('mlp', <keras.wrappers.scikit_learn.KerasClassifier object at 0x1d3b792438>)])

In [21]:
# Evaluate the validation set
score = pipeline.evaluate(x_val, y_val, verbose = 1)

AttributeError: 'Pipeline' object has no attribute 'evaluate'

In [ ]:
# Predictions
preds = pipeline.predict(x_test)

In [ ]:
# Results
print(confusion_matrix(y_test, preds))

print(classification_report(y_test, preds))

print(accuracy_score(y_test, preds))

---

## Conclusion:

At first, the task of creating a deep learning neural network might seem daunting. Here, I have tried to show that it really isnt all that difficult. 

As it turns out, our model was roughly 85% accurate. This isnt too bad considering that we quickly pieced together a simple toy example. 

In the future, to improve our results: we could play with the sizes of our hidden layers and also connect them to the output. The possibilities are endless. Feel free to toy with this example on your own.